In [1]:
# Clone the Bad Deed Validator repository
!git clone https://github.com/RitwijParmar/bad-deed-validator.git
%cd bad-deed-validator
print("Repository cloned successfully!")

Cloning into 'bad-deed-validator'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 12.85 KiB | 4.28 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/bad-deed-validator
Repository cloned successfully!


In [2]:
# Import the validator module
import sys
sys.path.insert(0, '/content/bad-deed-validator')

from validator import BadDeedValidator
import json

print("Validator module imported successfully!")

Validator module imported successfully!


In [3]:
# Test with the provided OCR deed text from the task
raw_deed_text = """*** RECORDING REQ ***
Doc: DEED-TRUST-0042
County: S. Clara | State: CA
Date Signed: 2024-01-15
Date Recorded: 2024-01-10
Grantor: T.E.S.L.A. Holdings LLC
Grantee: John & Sarah Connor
Amount: $1,250,000.00 (One Million Two Hundred Thousand Dollars)
APN: 992-001-XA
Status: PRELIMINARY
*** END ***"""

# Initialize validator
validator = BadDeedValidator('/content/bad-deed-validator/counties.json')

# Process the deed
report = validator.process_and_report(raw_deed_text)

print("="*80)
print("BAD DEED VALIDATOR - TEST RESULTS")
print("="*80)
print(f"\nStatus: {report['status']}")
print(f"\nValidation Summary:")
print(f"  - Errors: {report['validation_summary']['error_count']}")
print(f"  - Warnings: {report['validation_summary']['warning_count']}")

if report['validation_summary']['errors']:
    print(f"\nErrors Detected:")
    for i, error in enumerate(report['validation_summary']['errors'], 1):
        print(f"  [{i}] {error}")

if report['validation_summary']['warnings']:
    print(f"\nWarnings:")
    for i, warning in enumerate(report['validation_summary']['warnings'], 1):
        print(f"  [{i}] {warning}")

print(f"\nExtracted & Validated Data:")
deed_data = report['deed']
for key, value in deed_data.items():
    if key not in ['errors', 'warnings']:
        print(f"  {key}: {value}")

print("\n" + "="*80)

BAD DEED VALIDATOR - TEST RESULTS

Status: APPROVED

Validation Summary:
  - Errors: 0
  - Warnings: 2

Warnings:
  [1] Could not parse: One Million Two Hundred Thousand Dollars
  [2] Low confidence county match: 74%

Extracted & Validated Data:
  doc_id: DEED-TRUST-0042
  county_raw: S. Clara
  county_normalized: Santa Clara
  state: CA
  date_signed: 2024-01-15T00:00:00
  date_recorded: 2024-01-10T00:00:00
  grantor: T.E.S.L.A. Holdings LLC
  grantee: John & Sarah Connor
  amount_numeric: 1250000.0
  amount_written: One Million Two Hundred Thousand Dollars
  apn: 992-001-XA
  status: PRELIMINARY
  tax_rate: 0.012



In [4]:
# Test Case 2: REJECTED - Date Logic Error (recorded BEFORE signed)
print("\n" + "="*80)
print("TEST CASE 2: DATE LOGIC VIOLATION")
print("="*80)

bad_deed_text = """*** RECORDING REQ ***
Doc: DEED-FRAUD-0099
County: Santa Clara | State: CA
Date Signed: 2024-02-15
Date Recorded: 2024-02-10
Grantor: Fraudster Inc
Grantee: Innocent Buyer
Amount: $500,000.00 (Five Hundred Thousand Dollars)
APN: 123-456-XY
Status: PRELIMINARY
*** END ***"""

report2 = validator.process_and_report(bad_deed_text)
print(f"\nStatus: {report2['status']}")
print(f"Errors: {report2['validation_summary']['error_count']}")
print(f"Warnings: {report2['validation_summary']['warning_count']}")

if report2['validation_summary']['errors']:
    print(f"\nCRITICAL ERRORS DETECTED:")
    for i, error in enumerate(report2['validation_summary']['errors'], 1):
        print(f"\n  ERROR [{i}]:")
        print(f"  {error}")


TEST CASE 2: DATE LOGIC VIOLATION

Status: APPROVED
Errors: 0
Warnings: 1


In [5]:
# Test Case 3: REJECTED - Correct Date Logic Error (recorded BEFORE signed)
print("\n" + "="*80)
print("TEST CASE 3: DATE LOGIC VIOLATION (RECORDED 2024-01-10 BEFORE SIGNED 2024-01-15)")
print("="*80)

bad_deed_text2 = """*** RECORDING REQ ***
Doc: DEED-FRAUD-1234
County: Santa Clara | State: CA
Date Signed: 2024-03-15
Date Recorded: 2024-03-10
Grantor: Fraudster Corp
Grantee: Victim LLC
Amount: $2,000,000.00 (Two Million Dollars)
APN: 999-888-ZZ
Status: PRELIMINARY
*** END ***"""

report3 = validator.process_and_report(bad_deed_text2)
print(f"\nStatus: {report3['status']}")
print(f"Errors: {report3['validation_summary']['error_count']}")
print(f"Warnings: {report3['validation_summary']['warning_count']}")

if report3['validation_summary']['errors']:
    print(f"\n✗ CRITICAL ERRORS DETECTED:")
    for i, error in enumerate(report3['validation_summary']['errors'], 1):
        print(f"  [{i}] {error}")
else:
    print("\n✓ No errors detected")

if report3['validation_summary']['warnings']:
    print(f"\n⚠ WARNINGS:")
    for i, warning in enumerate(report3['validation_summary']['warnings'], 1):
        print(f"  [{i}] {warning}")


TEST CASE 3: DATE LOGIC VIOLATION (RECORDED 2024-01-10 BEFORE SIGNED 2024-01-15)

Status: APPROVED
Errors: 0
Warnings: 1

✓ No errors detected

⚠ WARNINGS:
  [1] Could not parse: Two Million Dollars
